In [3]:

from datasets import load_dataset # huggingface datasets
dataset = load_dataset("openwebtext", num_proc=60 , cache_dir="./.cache")

/home/angelo/autoregressive/.venv/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Setting num_proc from 60 to 21 for the train split as it only contains 21 shards.
Generating train split: 100%|██████████| 8013769/8013769 [01:33<00:00, 85871.37 examples/s] 


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer= AutoTokenizer.from_pretrained("gpt2", cache_dir="./.cache")
model = AutoModelForCausalLM.from_pretrained("gpt2", cache_dir="./.cache")


In [7]:
model.num_parameters()

124439808

In [6]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [3]:
import torch
from transformers import FalconModel
from torch.nn import functional as F

torch.manual_seed(0)

a = 3
b = 4

q = torch.ones(size=(1, 1, a, b))
k = torch.ones(size=(1, 1, a, b))
v = torch.ones(size=(1, 1, a, b))

def check(q, k, v, device):

    q = q.to(device)
    k = k.to(device)
    v = v.to(device)

    neg_value = torch.finfo(q.dtype).min
    mask = [[1, 1, 1], [1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]
    mask = torch.tensor([[mask]]).to(device)

    o = F.scaled_dot_product_attention(q, k, v, mask, 0.0, is_causal=False)
    print(o)

check(q, k, v, "cpu")
check(q, k, v, "cuda")

tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]])
tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]], device='cuda:0')


In [13]:
import torch

# Define the original array
original_array = torch.tensor([1, 2, 3])

# Duplicate each element and concatenate them
duplicated_array = original_array.repeat_interleave(2)

print(duplicated_array)


tensor([1, 1, 2, 2, 3, 3])


In [18]:
import torch

k = 2  # Step size
matrix = torch.tril(torch.ones(3, 3, dtype=torch.bool), diagonal=0)
matrix.repeat_interleave(2, dim=1)

tensor([[ True,  True, False, False, False, False],
        [ True,  True,  True,  True, False, False],
        [ True,  True,  True,  True,  True,  True]])

In [10]:
import torch

def stepcase_attention(size):
    # Create an identity matrix of the specified size
    identity_matrix = torch.eye(size)
    
    # Create a lower triangular matrix with ones below the main diagonal
    lower_triangular = torch.tril(torch.ones(size, size))
    
    # Subtract the lower triangular matrix from the identity matrix
    stepcase_matrix = identity_matrix - lower_triangular
    
    return stepcase_matrix

# Example usage:
matrix_size = 4
output_matrix = stepcase_attention(matrix_size)
print(output_matrix)


tensor([[ 0.,  0.,  0.,  0.],
        [-1.,  0.,  0.,  0.],
        [-1., -1.,  0.,  0.],
        [-1., -1., -1.,  0.]])


In [6]:
import torch

x = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
x = torch.arange(1, 37).reshape(2, 18)
[x[:, i::2] for i in range(2)]
# [:, :, i::self.config.k_regressivity, :] for i, lm_head in enumerate(self.lm_heads)]

[tensor([[ 1,  3,  5,  7,  9, 11, 13, 15, 17],
         [19, 21, 23, 25, 27, 29, 31, 33, 35]]),
 tensor([[ 2,  4,  6,  8, 10, 12, 14, 16, 18],
         [20, 22, 24, 26, 28, 30, 32, 34, 36]])]

In [27]:
targets = torch.ones(2, 18)
k = 2
b = 1
t = 10 # seq len
n_embd = 1
vocab_size = 1
# b x (t + (k - 1))

x = torch.arange(0, b * t * vocab_size).reshape(b, t, vocab_size)
targets = torch.ones(b, t + ( k - 1))

 # (b, t, n_embd)
logits = torch.stack([x[:, i::k, :] for i in range(k)]) # (k, b, t, vocab_size)
stacked_targets = torch.stack([targets[:, i:i+t][:, i::k] for i in range(k)], ) # (k, b, t)
            

In [29]:
logits[0, 0]

tensor([[0],
        [2],
        [4],
        [6],
        [8]])